## In this section we implement MinHash LSH using shingles instead of indiviadual words 

In [20]:
import numpy as np
import csv
import pandas
from datasketch import MinHash, MinHashLSH

In [21]:
# Shingle generators 
# Arguments : Message string, shingle size {in words}
# Returns : All shingles formed with k words
def shingle_generator(message, k):
    message = message.split(' ')
    item_set = []
    for i in range(0, len(message)-k):
        shingle = ""
        for j in range(0, k):
            shingle = shingle + message[i+j] + " "
        item_set.append(shingle)
    return item_set

In [22]:
# dump the CSV file into a numpy array. 
csv_reader = pandas.read_csv('data/string_study.csv', delimiter=',', header=None)
final_arr = csv_reader.values
final_arr = np.array(final_arr)

In [23]:
# extract all the raw_messsages from the error logs
raw_msgs = final_arr[0:, -1]
raw_msgs.shape
# raw_msgs[1:2] 

(34445,)

### The shingles test case is shown on the "shingle.ipynb" notebook

In [24]:
# Generate the set of words of all raw messages
message_set = []
%time

for item in raw_msgs:
#     set_of_words = set(item.split(' '))
    # Shingle size is set to 3 words
    set_of_words = set(shingle_generator(item, 2))
    message_set.append(set_of_words)
message_set[1:2]
len(message_set)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


34445

In [25]:
# Generate MinHash instances for all messages
m = []
for i in range (0, len(message_set)):
    m_temp = MinHash(num_perm=128)
    m.append(m_temp)
len(m)

34445

In [26]:
i = 0
for message in message_set:
    for word in message:
        m[i].update(word.encode('utf-8'))
    i = i + 1

In [38]:
# Create an LSH index 
# %time
lsh = MinHashLSH(threshold=0, num_perm=128)
count = 0
for i in range (0, len(m)):
    string = "m" + str(i)
    lsh.insert(string, m[i])
    count += 1
print (count)

34445


In [39]:
# Find all the messages similar to the fist message with a specified threshold
%time
result = lsh.query(m[1])
print (raw_msgs[1])
len(result)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs
Rally tired waiting 1440.00 seconds for Server rally-9db3-aPln:176b6fd3-8a2d-4b60-a5bd-73575c4ad161 to become ('ACTIVE') current status BUILD


7717

In [36]:
%time
len(result)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


7717

In [37]:
with open('min_hash_lsh_shingles_128_pointzero1.txt', 'w') as f:
    for item in result:
        f.write("%s\n" % item)

In [31]:
"m0" in result

False

In [32]:
"m1" in result

True

In [33]:
"m14037" in result

True